<h1>ASL Detector Training</h1>
<p>Created by Alex Pereira</p>

<h2>Preprocessing</h2>

In [ ]:
# Import Libraries
import os
import numpy as np
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras.utils  import to_categorical
from   sklearn.model_selection import train_test_split

In [ ]:
# Gets variables for data collection
DATA_PATH    = os.path.join("MP_Data")
action_list  = np.load("actions.npy")  # List of gestures
num_videos   = 30  # Number of videos
video_frames = 30  # Frames per video

In [ ]:
# Creates a label map
label_map = {label:num for num, label in enumerate(action_list)}

In [ ]:
sequences, labels = [], []

# 
for action in action_list:
    # 
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # 
        window = []

        # 
        for num_frame in range(0, video_frames):
            frame = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(num_frame)))
            window.append(frame)

        #
        sequences.append(window)

        # 
        labels.append(label_map[action])

In [ ]:
# Creates a numpy array
X = np.array(sequences)

# Stores the labels in an array
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

<h2>Model Creation</h2>

In [ ]:
# Import model building components
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Makes a sequential model
model = Sequential()

In [ ]:
# Adds layers to the model
model.add(LSTM(64,  return_sequences = True , activation = "relu", input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True , activation = "relu"))
model.add(LSTM(64,  return_sequences = False, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(action_list.shape[0], activation = "softmax"))

# Print a model summary
model.summary()

In [ ]:
# Defines the optimizer
customOptimizer = keras.optimizers.Adam(1e-4)

# Defines the loss function
customLoss      = keras.losses.CategoricalCrossentropy()

# Defines the callbacks to include
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath = "asl_accuracy/asl_detection.acc.{epoch:02d}-{cat_accuracy:.2f}.h5", monitor = "val_cat_accuracy", mode = "max", save_best_only = True),
    tf.keras.callbacks.ModelCheckpoint(filepath = "asl_loss/asl_detection.loss.{epoch:02d}-{val_loss:.2f}.h5", monitor = "loss", mode = "min", save_best_only = True),
    tf.keras.callbacks.EarlyStopping(patience = 500, monitor = "cat_accuracy"),
    tf.keras.callbacks.TensorBoard(log_dir  = "Logs")
]

# Creates customized metrics
custom_metrics = [
    tf.keras.metrics.CategoricalAccuracy(name = "cat_accuracy", dtype = None)
]

# Compile the model
model.compile(optimizer = customOptimizer, loss = customLoss, metrics = custom_metrics)

In [ ]:
# Trains the model
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5000, callbacks = my_callbacks, use_multiprocessing = True)

<h2>Accuracy Evaluation</h2>

In [ ]:
# Necesary imports
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
# Obtain some predictions
yhat = model.predict(X_test)

In [ ]:
# Interpret the predictions
ytrue = np.argmax(y_test, axis = 1).tolist()
yhat  = np.argmax(yhat  , axis = 1).tolist()

In [ ]:
# Create the confusion matrix
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
# Assess accuracy
accuracy_score(ytrue, yhat)